In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


Every `nn.Module` subclass implements the operations on input data in the `forward` method.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3])


# Model layers

Let’s break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


We initialize the `nn.Flatten` layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).

In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In this model, we use `nn.ReLU` between our linear layers, but there’s other activations to introduce non-linearity in your model.

In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0986, -0.1875,  0.1223,  0.0047,  0.7945, -0.0912, -0.1398, -0.1381,
          0.4536, -0.0132, -0.0161, -0.3609,  0.4279, -0.1424,  0.0410, -0.1806,
         -0.0073,  0.7960, -0.2426, -0.5957],
        [ 0.1616, -0.1600,  0.2577, -0.2159,  0.3183, -0.2267, -0.5087,  0.3199,
          0.2663, -0.3669, -0.0218, -0.3856,  0.1623,  0.1911,  0.1425,  0.0581,
         -0.2493,  0.1678, -0.1281, -0.5676],
        [ 0.1371, -0.3054, -0.1924,  0.1823,  0.8747,  0.0571, -0.2462, -0.0652,
          0.4518, -0.4106,  0.2374, -0.5397,  0.3861, -0.1363,  0.1723, -0.0711,
         -0.1047,  0.5133, -0.1705, -0.3293]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.1223, 0.0047, 0.7945, 0.0000, 0.0000, 0.0000, 0.4536,
         0.0000, 0.0000, 0.0000, 0.4279, 0.0000, 0.0410, 0.0000, 0.0000, 0.7960,
         0.0000, 0.0000],
        [0.1616, 0.0000, 0.2577, 0.0000, 0.3183, 0.0000, 0.0000, 0.3199, 0.2663,
         0.0000, 0.0000, 0.0000, 0.1623, 0.1911, 0.14

`nn.Sequential` is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like `seq_modules`.

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

The last linear layer of the neural network returns *logits* - raw values in [-infty, infty] - which are passed to the `nn.Softmax` module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. `dim` parameter indicates the dimension along which the values must sum to 1.

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

# Model parameters

Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s `parameters()` or `named_parameters()` methods.

In [13]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0047,  0.0127, -0.0037,  ...,  0.0347,  0.0262, -0.0219],
        [ 0.0277,  0.0088, -0.0334,  ..., -0.0299, -0.0019,  0.0079]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0300, 0.0223], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0170,  0.0131,  0.0352,  ...,  0.0054,  0.0245, -0.0346],
        [-0.0020,  0.0363,  0.0167,  ...,  0.0033, -0.0178,  0.0383]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Si